In [1]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def find_sheet_name(sheet_names):
    cpi_sheet_names = list(filter(lambda x: "fdi" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    cpi_sheet_names = list(filter(lambda x: "dtnn" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    if len(sheet_names) == 43:
        return "22"

    cpi_sheet_names = list(filter(lambda x: "sheet1" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    raise RuntimeError(sheet_names, len(sheet_names))

In [3]:
en_to_vn = {
    "TOTAL": "TỔNG SỐ",
    "By province": "Phân theo một số địa phương",
    "Bac Lieu": "Bạc Liêu",
    "Hai Phong": "Hải Phòng",
    "Binh Duong": "Bình Dương",
    "Ha Nam": "Hà Nam",
    "Ha Noi": "Hà Nội",
    "Ho Chi Minh city": "TP. Hồ Chí Minh",
    "Tay Ninh": "Tây Ninh",
    "Bac Ninh": "Bắc Ninh",
    "Quang Ninh": "Quảng Ninh",
    "Bac Giang": "Bác Giang",
    "Quang Binh": "Quảng Bình",
    "Ba Ria - Vung Tau": "Bà Rịa - Vũng Tàu",
    "Long An": "Long An",
    "Thanh Hoa": "Thanh Hóa",
    "Dong Nai": "Đồng Nai",
    "Phu Tho": "Phú Thọ",
    "Da Nang": "Đà Nẵng",
    "Hai Duong": "Hải Dương",
    "Vinh Phuc": "Vĩnh Phúc",
    "Vinh Long": "Vĩnh Long",
    "By country and geographical territory": "Phân theo một số nước và vùng lãnh thổ",
    "Singapore": "Xin-ga-po",
    "China": "Trung Quốc",
    "South Korea": "Hàn Quốc",
    "Special Administration Hong Kong": "Đặc khu HC Hồng Công (TQ)",
    "Taiwan": "Đài Loan",
    "Japan": "Nhật Bản",
    "Philippines": "Phi-li-pin",
    "BritishVirginIslands": "Quần đảo Virgin thuộc Anh",
    "Thailand": "Thái Lan",
    "Netherlands": "Hà Lan",
    "Cayman Islands": "Quần đảo Cay-man",
    "Samoa": "Xa-moa",
    "United State": "Hoa Kỳ",
    "Seychelles": "Xây-Sen",
    "Malaysia": "Ma-lai-xi-a",
    "France": "Pháp",
    "Germany": "Đức",
}

In [4]:
from datetime import timedelta


def get_cpi(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Phân theo một số địa phương")
                | (df[df.columns[1]] == "By province")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "TP. Hồ Chí Minh")
                | (df[df.columns[1]] == "Ho Chi Minh city")
                | (df[df.columns[1]] == " TP. Hồ Chí Minh ")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
        .str.strip()
    )

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "TỔNG SỐ") | ((df[df.columns[0]] == "TOTAL"))
                ].index[0]
            )
        )
    )

    if len(df[df.columns[0]][df[df.columns[0]] == "TOTAL"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    df = df.dropna(axis="columns", how="all")
    df = df.dropna(axis="index", how="all")

    if len(df.columns) == 4:
        df.columns = [
            "Phân Loại",
            "Số dự án",
            "Vốn đăng ký cấp mới",
            "Vốn đăng ký điều chỉnh",
        ]

        df = df[pd.to_numeric(df["Số dự án"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["Vốn đăng ký cấp mới"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["Vốn đăng ký điều chỉnh"], errors="coerce").notnull()]

        df["Vốn đăng ký"] = df["Vốn đăng ký cấp mới"] + df["Vốn đăng ký điều chỉnh"]
    elif len(df.columns) == 3:
        df.columns = [
            "Phân Loại",
            "Số dự án",
            "Vốn đăng ký",
        ]

        df = df[pd.to_numeric(df["Số dự án"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["Vốn đăng ký"], errors="coerce").notnull()]
    else:
        raise RuntimeError

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    df["Month"] = df["Date"].dt.strftime("%Y-%m")
    df["PreMonth"] = (
        df["Date"]
        .apply(
            lambda x: (
                x.replace(month=x.month - 1, day=1)
                if x.month > 1
                else x.replace(year=x.year - 1, month=12, day=1)
            )
        )
        .dt.strftime("%Y-%m")
    )

    df["PreYearMonth"] = (
        df["Date"]
        .apply(lambda x: x.replace(year=x.year - 1, day=1))
        .dt.strftime("%Y-%m")
    )

    df = df[~df["Phân Loại"].isnull()]

    return df


# xl = pd.ExcelFile("../datas/2024-03-29-02-Bieu-3.2024-ngay-28.3.xlsx")

# sheet_name = find_sheet_name(xl.sheet_names)

# get_cpi(xl, sheet_name, "2024-03-29")['Phân Loại'].unique().tolist()

In [5]:
import os
import contextlib

dfs_fdi_origin = []
for file_name in sorted(
    os.listdir("../datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("../datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2016:
            continue

        xl = pd.ExcelFile(file_path)

        df_fdi = get_cpi(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_fdi_origin.append(df_fdi)
    except Exception as e:
        print(file_name)
        raise e

In [6]:
dfs_fdi = pd.concat(dfs_fdi_origin, ignore_index=True, axis=0)
dfs_fdi["Date"] = pd.to_datetime(dfs_fdi["Date"], dayfirst=False)
dfs_fdi

,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date,Month,PreMonth,PreYearMonth
0,TỔNG SỐ,2492,13553.757566,7643.956388,21197.713954,2024-09-29,2024-09,2024-08,2023-09
1,Bà Rịa - Vũng Tàu,34,1681.168637,-9.557457,1671.61118,2024-09-29,2024-09,2024-08,2023-09
2,Bắc Ninh,325,1613.384153,2644.33862,4257.722773,2024-09-29,2024-09,2024-08,2023-09
3,Quảng Ninh,27,1609.642987,203.00106,1812.644047,2024-09-29,2024-09,2024-08,2023-09
4,Hà Nội,208,1121.315759,179.449427,1300.765185,2024-09-29,2024-09,2024-08,2023-09
...,...,...,...,...,...,...,...,...,...
3769,Quần đảo Vigin thuộc Anh,3,NaN,NaN,9.4,2016-01-29,2016-01,2015-12,2015-01
3770,Đài Loan,6,NaN,NaN,8.7,2016-01-29,2016-01,2015-12,2015-01
3771,Thái Lan,3,NaN,NaN,6.3,2016-01-29,2016-01,2015-12,2015-01
3772,Hà Lan,1,NaN,NaN,6,2016-01-29,2016-01,2015-12,2015-01


In [7]:
convert_pl = {
    "TỔNG SỐ": "TỔNG SỐ",
    "Hà Nội": "Hà Nội",
    "Quảng Ninh": "Quảng Ninh",
    "Bắc Ninh": "Bắc Ninh",
    "Thái Nguyên": "Thái Nguyên",
    "Hưng Yên": "Hưng Yên",
    "Bà Rịa - Vũng Tàu": "Bà Rịa - Vũng Tàu",
    "Đồng Nai": "Đồng Nai",
    "Hải Phòng": "Hải Phòng",
    "Long An": "Long An",
    "Bình Dương": "Bình Dương",
    "TP. Hồ Chí Minh": "TP. Hồ Chí Minh",
    "Bắc Giang": "Bắc Giang",
    "Thái Bình": "Thái Bình",
    "Nam Định": "Nam Định",
    "Hà Nam": "Hà Nam",
    "Vĩnh Phúc": "Vĩnh Phúc",
    "Nghệ An": "Nghệ An",
    "Xin-ga-po": "Xin-ga-po",
    "Đặc khu hành chính Hồng Kông (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Trung Quốc": "Trung Quốc",
    "Nhật Bản": "Nhật Bản",
    "Hàn Quốc": "Hàn Quốc",
    "Đài Loan": "Đài Loan",
    "Vương quốc Anh": "Vương quốc Anh",
    "Xa-moa": "Xa-moa",
    "I-ta-li-a": "I-ta-li-a",
    "Thái Lan": "Thái Lan",
    "Xây-xen": "Xây-Sen",
    "Hoa Kỳ": "Hoa Kỳ",
    "Ma-lai-xi-a": "Ma-lai-xi-a",
    "Quần đảo Vigin thuộc Anh": "Quần đảo Vigin thuộc Anh",
    "Thanh Hóa": "Thanh Hóa",
    "Thừa Thiên Huế": "Thừa Thiên - Huế",
    "Ninh Bình": "Ninh Bình",
    "Đà Nẵng": "Đà Nẵng",
    "Ca-na-da": "Ca-na-da",
    "Pháp": "Pháp",
    "Phần Lan": "Phần Lan",
    "Na-uy": "Na-uy",
    "Yên Bái": "Yên Bái",
    "Hải Dương": "Hải Dương",
    "Phú Thọ": "Phú Thọ",
    "Bình Phước": "Bình Phước",
    "Đặc khu HC Hồng Công (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Đức": "Đức",
    "Hà Lan": "Hà Lan",
    "Xa-Moa": "Xa-moa",
    "Thụy Điển": "Thụy Điển",
    "Xây-Sen": "Xây-Sen",
    "Đan Mạch": "Đan Mạch",
    "Ấn Độ": "Ấn Độ",
    "Quần đảo Virgin thuộc Anh": "Quần đảo Vigin thuộc Anh",
    "I-xa-ren": "I-xa-ren",
    "Tây Ninh": "Tây Ninh",
    "Quảng Ngãi": "Quảng Ngãi",
    "I-xa_ren": "I-xa-ren",
    "Đặc khu hành chính Hồng Công (Trung Quốc)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Xây- Sen": "Xây-Sen",
    "Sóc Trăng": "Sóc Trăng",
    "Đăk Lăk": "Đăk Lăk",
    "Đặc khu hành chính Hồng Công (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Thụy Sỹ": "Thụy Sỹ",
    "Xây-sen": "Xây-Sen",
    "Hà Tĩnh": "Hà Tĩnh",
    "Béc-mu-da": "Béc-mu-da",
    "Cần Thơ": "Cần Thơ",
    "CHLB Đức": "Đức",
    "Béc-mu-đa": "Béc-mu-da",
    "Quảng Nam": "Quảng Nam",
    "Bỉ": "Bỉ",
    "Xây Sen": "Xây-Sen",
    "Ôx-trây-li-a": "Ôx-trây-li-a",
    "Quần đảo Virgin thuộc Mỹ": "Quần đảo Vigin thuộc Anh",
    "Phi-li-pin": "Phi-li-pin",
    "Phi-lip-pin": "Phi-li-pin",
    "Hậu Giang": "Hậu Giang",
    "Quần đảo Cay-man": "Quần đảo Cay-man",
    "Ô-xtrây-li-a": "Ôx-trây-li-a",
    "Quần đảo Cây- men": "Quần đảo Cay-man",
    "Bến Tre": "Bến Tre",
    "Quảng Bình": "Quảng Bình",
    "Bác Giang": "Bắc Giang",
    "Vĩnh Long": "Vĩnh Long",
    "Tiền Giang": "Tiền Giang",
    "Bình Thuận": "Bình Thuận",
    "Hồng Công": "Đặc khu hành chính Hồng Kông (TQ)",
    "Lúc-xăm-bua": "Lúc-xăm-bua",
    "Thừa Thiên - Huế": "Thừa Thiên - Huế",
    "Đặc khu Hành chính Hồng Công (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "CHND Trung Hoa": "Trung Quốc",
    "Niu Di-lân": "Niu Di-lân",
    "Luxembourg": "Lúc-xăm-bua",
    "Anguilla": "Anguilla",
    "Hồng Kông": "Đặc khu hành chính Hồng Kông (TQ)",
    "Phú Yên": "Phú Yên",
    "Ca-na-đa": "Ca-na-da",
    "Quần đảo Cay-men": "Quần đảo Cay-man",
    "Bê-li-xê": "Bê-li-xê",
    "Thừa Thiên - Huế": "Thừa Thiên - Huế",
    "Thanh Hóa": "Thanh Hóa",
    "Vĩnh Long": "Vĩnh Long",
    "Bến Tre": "Bến Tre",
    "Đà Nẵng": "Đà Nẵng",
    "Thái Lan": "Thái Lan",
    "Quần đảo Cay-men": "Quần đảo Cay-man",
    "Đan Mạch": "Đan Mạch",
    "Cộng hòa Síp": "Cộng hòa Síp",
    "Ninh Thuận": "Ninh Thuận",
    "Thái Nguyên": "Thái Nguyên",
    "Bạc Liêu": "Bạc Liêu",
    "Quảng Ngãi": "Quảng Ngãi",
    "Kiên Giang": "Kiên Giang",
    "Vương quốc Bỉ": "Vương quốc Bỉ",
    "Ma-cao": "Ma-cao",
    "In-đô-nê-xi-a": "In-đô-nê-xi-a",
    "Na Uy": "Na-uy",
    "Trà Vinh": "Trà Vinh",
    "Bơ-mu-đa": "Bơ-mu-đa",
    "Thụy Sĩ": "Thụy Sỹ",
    "Các Tiểu vương quốc A-rập Thống nhất": "Các Tiểu vương quốc A-rập Thống nhất",
    "Đắk Lắk": "Đăk Lăk",
    "Pa-ki-xtan": "Pa-ki-xtan",
    "Khánh Hòa": "Khánh Hòa",
    "Ô-man": "Ô-man",
    "Tây Ấn thuộc Anh": "Tây Ấn thuộc Anh",
    "Trà Vinh": "Trà Vinh",
    "Hung-ga-ri": "Hung-ga-ri",
    "Xlô-va-ki": "Xlô-va-ki",
    "Xlô-va-ki-a": "Xlô-va-ki",
    "Đắc Lắk": "Đăk Lăk",
    "Lâm Đồng": "Lâm Đồng",
    "Ni-giê-ri-a": "Ni-giê-ri-a",
    "Bác-ba-đốt": "Bác-ba-đốt",
    "Bru-nây": "Bru-nây",
    "Đặc khu Hành chính Hồng Kông (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Đặ khu HC Hồng Kông (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Thổ Nhĩ Kỳ": "Thổ Nhĩ Kỳ",
    "Quần đảo Cây-men": "Quần đảo Cay-man",
}

for pl in dfs_fdi["Phân Loại"].unique().tolist():
    assert pl in convert_pl.keys(), pl


dfs_fdi["Phân Loại"] = dfs_fdi["Phân Loại"].map(convert_pl)

In [8]:
convert_to_mien = {
    "TỔNG SỐ": "TỔNG SỐ",
    "Lâm Đồng": "Tây Nguyên",
    "Đăk Lăk": "Tây Nguyên",
    "Bắc Giang": "Miền Bắc",
    "Bắc Ninh": "Miền Bắc",
    "Hà Nam": "Miền Bắc",
    "Hà Nội": "Miền Bắc",
    "Hưng Yên": "Miền Bắc",
    "Hải Dương": "Miền Bắc",
    "Hải Phòng": "Miền Bắc",
    "Nam Định": "Miền Bắc",
    "Nghệ An": "Miền Bắc",
    "Ninh Bình": "Miền Bắc",
    "Phú Thọ": "Miền Bắc",
    "Phú Yên": "Miền Bắc",
    "Quảng Ninh": "Miền Bắc",
    "Thanh Hóa": "Miền Bắc",
    "Thái Bình": "Miền Bắc",
    "Thái Nguyên": "Miền Bắc",
    "Vĩnh Phúc": "Miền Bắc",
    "Yên Bái": "Miền Bắc",
    "Bến Tre": "Miền Nam",
    "Bà Rịa - Vũng Tàu": "Miền Nam",
    "Bình Dương": "Miền Nam",
    "Bình Phước": "Miền Nam",
    "Bạc Liêu": "Miền Nam",
    "Cần Thơ": "Miền Nam",
    "Hậu Giang": "Miền Nam",
    "Kiên Giang": "Miền Nam",
    "Long An": "Miền Nam",
    "Sóc Trăng": "Miền Nam",
    "TP. Hồ Chí Minh": "Miền Nam",
    "Tiền Giang": "Miền Nam",
    "Trà Vinh": "Miền Nam",
    "Tây Ninh": "Miền Nam",
    "Vĩnh Long": "Miền Nam",
    "Đồng Nai": "Miền Nam",
    "Bình Thuận": "Miền Trung",
    "Hà Tĩnh": "Miền Trung",
    "Khánh Hòa": "Miền Trung",
    "Ninh Thuận": "Miền Trung",
    "Quảng Ngãi": "Miền Trung",
    "Quảng Bình": "Miền Trung",
    "Quảng Nam": "Miền Trung",
    "Thừa Thiên - Huế": "Miền Trung",
    "Đà Nẵng": "Miền Trung",
    "In-đô-nê-xi-a": "Châu Á",
    "Nhật Bản": "Châu Á",
    "Trung Quốc": "Châu Á",
    "Đài Loan": "Châu Á",
    "Đặc khu hành chính Hồng Kông (TQ)": "Châu Á",
    "Ấn Độ": "Châu Á",
    "Hàn Quốc": "Châu Á",
    "Thái Lan": "Châu Á",
    "Phi-li-pin": "Châu Á",
    "Các Tiểu vương quốc A-rập Thống nhất": "Châu Á",
    "Bru-nây": "Châu Á",
    "I-xa-ren": "Châu Á",
    "Ma-cao": "Châu Á",
    "Ma-lai-xi-a": "Châu Á",
    "Pa-ki-xtan": "Châu Á",
    "Ô-man": "Châu Á",
    "Xin-ga-po": "Châu Á",
    "Bỉ": "Châu Âu",
    "Na-uy": "Châu Âu",
    "Pháp": "Châu Âu",
    "Phần Lan": "Châu Âu",
    "Thụy Sỹ": "Châu Âu",
    "Thụy Điển": "Châu Âu",
    "Vương quốc Anh": "Châu Âu",
    "Vương quốc Bỉ": "Châu Âu",
    "Đan Mạch": "Châu Âu",
    "Đức": "Châu Âu",
    "Hà Lan": "Châu Âu",
    "Cộng hòa Síp": "Châu Âu",
    "Hung-ga-ri": "Châu Âu",
    "I-ta-li-a": "Châu Âu",
    "Lúc-xăm-bua": "Châu Âu",
    "Xlô-va-ki": "Châu Âu",
    "Anguilla": "Châu Mỹ",
    "Bác-ba-đốt": "Châu Mỹ",
    "Béc-mu-da": "Châu Mỹ",
    "Bê-li-xê": "Châu Mỹ",
    "Bơ-mu-đa": "Châu Mỹ",
    "Ca-na-da": "Châu Mỹ",
    "Quần đảo Cay-man": "Châu Mỹ",
    "Quần đảo Vigin thuộc Anh": "Châu Mỹ",
    "Tây Ấn thuộc Anh": "Châu Mỹ",
    "Hoa Kỳ": "Châu Mỹ",
    "Ni-giê-ri-a": "Châu Phi",
    "Xây-Sen": "Châu Phi",
    "Niu Di-lân": "Châu Úc",
    "Xa-moa": "Châu Úc",
    "Ôx-trây-li-a": "Châu Úc",
    "Thổ Nhĩ Kỳ": "Châu Âu",
    "Quần đảo Cây-men": "Châu Mỹ",
}


for pl in dfs_fdi["Phân Loại"].unique().tolist():
    assert pl in convert_to_mien.keys(), pl

dfs_fdi["Vùng Miền"] = dfs_fdi["Phân Loại"].map(convert_to_mien)

In [9]:
convert_to_mien = {
    "TỔNG SỐ": "TỔNG SỐ",
    "Lâm Đồng": "Trong Nước",
    "Đăk Lăk": "Trong Nước",
    "Bắc Giang": "Trong Nước",
    "Bắc Ninh": "Trong Nước",
    "Hà Nam": "Trong Nước",
    "Hà Nội": "Trong Nước",
    "Hưng Yên": "Trong Nước",
    "Hải Dương": "Trong Nước",
    "Hải Phòng": "Trong Nước",
    "Nam Định": "Trong Nước",
    "Nghệ An": "Trong Nước",
    "Ninh Bình": "Trong Nước",
    "Phú Thọ": "Trong Nước",
    "Phú Yên": "Trong Nước",
    "Quảng Ninh": "Trong Nước",
    "Thanh Hóa": "Trong Nước",
    "Thái Bình": "Trong Nước",
    "Thái Nguyên": "Trong Nước",
    "Vĩnh Phúc": "Trong Nước",
    "Yên Bái": "Trong Nước",
    "Bến Tre": "Trong Nước",
    "Bà Rịa - Vũng Tàu": "Trong Nước",
    "Bình Dương": "Trong Nước",
    "Bình Phước": "Trong Nước",
    "Bạc Liêu": "Trong Nước",
    "Cần Thơ": "Trong Nước",
    "Hậu Giang": "Trong Nước",
    "Kiên Giang": "Trong Nước",
    "Long An": "Trong Nước",
    "Sóc Trăng": "Trong Nước",
    "TP. Hồ Chí Minh": "Trong Nước",
    "Tiền Giang": "Trong Nước",
    "Trà Vinh": "Trong Nước",
    "Tây Ninh": "Trong Nước",
    "Vĩnh Long": "Trong Nước",
    "Đồng Nai": "Trong Nước",
    "Bình Thuận": "Trong Nước",
    "Hà Tĩnh": "Trong Nước",
    "Khánh Hòa": "Trong Nước",
    "Ninh Thuận": "Trong Nước",
    "Quảng Ngãi": "Trong Nước",
    "Quảng Bình": "Trong Nước",
    "Quảng Nam": "Trong Nước",
    "Thừa Thiên - Huế": "Trong Nước",
    "Đà Nẵng": "Trong Nước",
    "In-đô-nê-xi-a": "Nước Ngoài",
    "Nhật Bản": "Nước Ngoài",
    "Trung Quốc": "Nước Ngoài",
    "Đài Loan": "Nước Ngoài",
    "Đặc khu hành chính Hồng Kông (TQ)": "Nước Ngoài",
    "Ấn Độ": "Nước Ngoài",
    "Hàn Quốc": "Nước Ngoài",
    "Thái Lan": "Nước Ngoài",
    "Phi-li-pin": "Nước Ngoài",
    "Các Tiểu vương quốc A-rập Thống nhất": "Nước Ngoài",
    "Bru-nây": "Nước Ngoài",
    "I-xa-ren": "Nước Ngoài",
    "Ma-cao": "Nước Ngoài",
    "Ma-lai-xi-a": "Nước Ngoài",
    "Pa-ki-xtan": "Nước Ngoài",
    "Ô-man": "Nước Ngoài",
    "Xin-ga-po": "Nước Ngoài",
    "Bỉ": "Nước Ngoài",
    "Na-uy": "Nước Ngoài",
    "Pháp": "Nước Ngoài",
    "Phần Lan": "Nước Ngoài",
    "Thụy Sỹ": "Nước Ngoài",
    "Thụy Điển": "Nước Ngoài",
    "Vương quốc Anh": "Nước Ngoài",
    "Vương quốc Bỉ": "Nước Ngoài",
    "Đan Mạch": "Nước Ngoài",
    "Đức": "Nước Ngoài",
    "Hà Lan": "Nước Ngoài",
    "Cộng hòa Síp": "Nước Ngoài",
    "Hung-ga-ri": "Nước Ngoài",
    "I-ta-li-a": "Nước Ngoài",
    "Lúc-xăm-bua": "Nước Ngoài",
    "Xlô-va-ki": "Nước Ngoài",
    "Anguilla": "Nước Ngoài",
    "Bác-ba-đốt": "Nước Ngoài",
    "Béc-mu-da": "Nước Ngoài",
    "Bê-li-xê": "Nước Ngoài",
    "Bơ-mu-đa": "Nước Ngoài",
    "Ca-na-da": "Nước Ngoài",
    "Quần đảo Cay-man": "Nước Ngoài",
    "Quần đảo Vigin thuộc Anh": "Nước Ngoài",
    "Tây Ấn thuộc Anh": "Nước Ngoài",
    "Hoa Kỳ": "Nước Ngoài",
    "Ni-giê-ri-a": "Nước Ngoài",
    "Xây-Sen": "Nước Ngoài",
    "Niu Di-lân": "Nước Ngoài",
    "Xa-moa": "Nước Ngoài",
    "Ôx-trây-li-a": "Nước Ngoài",
    "Thổ Nhĩ Kỳ": "Châu Âu",
    "Quần đảo Cây-men": "Châu Mỹ",
}


for pl in dfs_fdi["Phân Loại"].unique().tolist():
    assert pl in convert_to_mien.keys(), pl

dfs_fdi["Trong Nước"] = dfs_fdi["Phân Loại"].map(convert_to_mien)

In [10]:
dfs_fdi

,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date,Month,PreMonth,PreYearMonth,Vùng Miền,Trong Nước
0,TỔNG SỐ,2492,13553.757566,7643.956388,21197.713954,2024-09-29,2024-09,2024-08,2023-09,TỔNG SỐ,TỔNG SỐ
1,Bà Rịa - Vũng Tàu,34,1681.168637,-9.557457,1671.61118,2024-09-29,2024-09,2024-08,2023-09,Miền Nam,Trong Nước
2,Bắc Ninh,325,1613.384153,2644.33862,4257.722773,2024-09-29,2024-09,2024-08,2023-09,Miền Bắc,Trong Nước
3,Quảng Ninh,27,1609.642987,203.00106,1812.644047,2024-09-29,2024-09,2024-08,2023-09,Miền Bắc,Trong Nước
4,Hà Nội,208,1121.315759,179.449427,1300.765185,2024-09-29,2024-09,2024-08,2023-09,Miền Bắc,Trong Nước
...,...,...,...,...,...,...,...,...,...,...,...
3769,Quần đảo Vigin thuộc Anh,3,NaN,NaN,9.4,2016-01-29,2016-01,2015-12,2015-01,Châu Mỹ,Nước Ngoài
3770,Đài Loan,6,NaN,NaN,8.7,2016-01-29,2016-01,2015-12,2015-01,Châu Á,Nước Ngoài
3771,Thái Lan,3,NaN,NaN,6.3,2016-01-29,2016-01,2015-12,2015-01,Châu Á,Nước Ngoài
3772,Hà Lan,1,NaN,NaN,6,2016-01-29,2016-01,2015-12,2015-01,Châu Âu,Nước Ngoài


In [11]:
merge_df = pd.merge(
    left=dfs_fdi,
    right=dfs_fdi,
    left_on=["Phân Loại", "PreMonth"],
    right_on=["Phân Loại", "Month"],
    how="left",
)
merge_df["Số dự án trong tháng"] = merge_df["Số dự án_x"] - merge_df["Số dự án_y"]
merge_df.loc[merge_df["Số dự án trong tháng"] < 0, "Số dự án trong tháng"] = merge_df[
    "Số dự án_x"
]

merge_df["Vốn đăng ký cấp mới trong tháng"] = (
    merge_df["Vốn đăng ký cấp mới_x"] - merge_df["Vốn đăng ký cấp mới_y"]
)
merge_df.loc[
    merge_df["Vốn đăng ký cấp mới trong tháng"] < 0, "Vốn đăng ký cấp mới trong tháng"
] = merge_df["Vốn đăng ký cấp mới_x"]

merge_df["Vốn đăng ký điều chỉnh trong tháng"] = (
    merge_df["Vốn đăng ký điều chỉnh_x"] - merge_df["Vốn đăng ký điều chỉnh_y"]
)
merge_df.loc[
    merge_df["Vốn đăng ký điều chỉnh trong tháng"] < 0,
    "Vốn đăng ký điều chỉnh trong tháng",
] = merge_df["Vốn đăng ký điều chỉnh_x"]

merge_df["Vốn đăng ký trong tháng"] = (
    merge_df["Vốn đăng ký_x"] - merge_df["Vốn đăng ký_y"]
)
merge_df.loc[merge_df["Vốn đăng ký trong tháng"] < 0, "Vốn đăng ký trong tháng"] = (
    merge_df["Vốn đăng ký_x"]
)

merge_df = merge_df.drop(
    columns=[
        "Số dự án_y",
        "Vốn đăng ký cấp mới_y",
        "Vốn đăng ký điều chỉnh_y",
        "Vốn đăng ký_y",
        "Date_y",
        "Month_y",
        "PreMonth_y",
        "PreYearMonth_y",
        "Vùng Miền_y",
        "Trong Nước_y",
    ]
)
merge_df = merge_df.rename(
    columns={
        "Số dự án_x": "Số dự án",
        "Vốn đăng ký cấp mới_x": "Vốn đăng ký cấp mới",
        "Vốn đăng ký điều chỉnh_x": "Vốn đăng ký điều chỉnh",
        "Vốn đăng ký_x": "Vốn đăng ký",
        "Date_x": "Date",
        "Month_x": "Month",
        "PreMonth_x": "PreMonth",
        "PreYearMonth_x": "PreYearMonth",
        "Vùng Miền_x": "Vùng Miền",
        "Trong Nước_x": "Trong Nước",
    }
)

dfs_fdi = merge_df
dfs_fdi

,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date,Month,PreMonth,PreYearMonth,Vùng Miền,Trong Nước,Số dự án trong tháng,Vốn đăng ký cấp mới trong tháng,Vốn đăng ký điều chỉnh trong tháng,Vốn đăng ký trong tháng
0,TỔNG SỐ,2492,13553.757566,7643.956388,21197.713954,2024-09-29,2024-09,2024-08,2023-09,TỔNG SỐ,TỔNG SỐ,245,1555.257566,1931.256388,3486.513954
1,Bà Rịa - Vũng Tàu,34,1681.168637,-9.557457,1671.61118,2024-09-29,2024-09,2024-08,2023-09,Miền Nam,Trong Nước,4,29.6,0.0,29.6
2,Bắc Ninh,325,1613.384153,2644.33862,4257.722773,2024-09-29,2024-09,2024-08,2023-09,Miền Bắc,Trong Nước,7,13.7,1028.574973,1042.274973
3,Quảng Ninh,27,1609.642987,203.00106,1812.644047,2024-09-29,2024-09,2024-08,2023-09,Miền Bắc,Trong Nước,2,35.063265,0.0,35.063265
4,Hà Nội,208,1121.315759,179.449427,1300.765185,2024-09-29,2024-09,2024-08,2023-09,Miền Bắc,Trong Nước,30,18.520513,15.365383,33.885896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3783,Quần đảo Vigin thuộc Anh,3,NaN,NaN,9.4,2016-01-29,2016-01,2015-12,2015-01,Châu Mỹ,Nước Ngoài,NaN,NaN,NaN,NaN
3784,Đài Loan,6,NaN,NaN,8.7,2016-01-29,2016-01,2015-12,2015-01,Châu Á,Nước Ngoài,NaN,NaN,NaN,NaN
3785,Thái Lan,3,NaN,NaN,6.3,2016-01-29,2016-01,2015-12,2015-01,Châu Á,Nước Ngoài,NaN,NaN,NaN,NaN
3786,Hà Lan,1,NaN,NaN,6,2016-01-29,2016-01,2015-12,2015-01,Châu Âu,Nước Ngoài,NaN,NaN,NaN,NaN


In [12]:
merge_df = pd.merge(
    left=dfs_fdi,
    right=dfs_fdi,
    left_on=["Phân Loại", "PreYearMonth"],
    right_on=["Phân Loại", "Month"],
    how="left",
)

merge_df["Số dự án trong tháng - YOY"] = np.where(
    merge_df["Số dự án trong tháng_y"] != 0,
    100
    * (
        merge_df["Số dự án trong tháng_x"]
        / (merge_df["Số dự án trong tháng_y"] + 1.0e-9)
    )
    - 100,
    np.nan,
)
merge_df["Vốn đăng ký cấp mới trong tháng - YOY"] = np.where(
    merge_df["Vốn đăng ký cấp mới trong tháng_y"] != 0,
    100
    * (
        merge_df["Vốn đăng ký cấp mới trong tháng_x"]
        / (merge_df["Vốn đăng ký cấp mới trong tháng_y"] + 1.0e-9)
    )
    - 100,
    np.nan,
)
merge_df["Vốn đăng ký điều chỉnh trong tháng - YOY"] = np.where(
    merge_df["Vốn đăng ký điều chỉnh trong tháng_y"] != 0,
    100
    * (
        merge_df["Vốn đăng ký điều chỉnh trong tháng_x"]
        / (merge_df["Vốn đăng ký điều chỉnh trong tháng_y"] + 1.0e-9)
    )
    - 100,
    np.nan,
)
merge_df["Vốn đăng ký trong tháng - YOY"] = np.where(
    merge_df["Vốn đăng ký trong tháng_y"] != 0,
    100
    * (
        merge_df["Vốn đăng ký trong tháng_x"]
        / (merge_df["Vốn đăng ký trong tháng_y"] + 1.0e-9)
    )
    - 100,
    np.nan,
)

merge_df = merge_df.drop(
    columns=[
        "Số dự án_y",
        "Vốn đăng ký cấp mới_y",
        "Vốn đăng ký điều chỉnh_y",
        "Vốn đăng ký_y",
        "Date_y",
        "Month_y",
        "PreMonth_y",
        "PreYearMonth_y",
        "Vùng Miền_y",
        "Trong Nước_y",
        "Số dự án trong tháng_y",
        "Vốn đăng ký cấp mới trong tháng_y",
        "Vốn đăng ký điều chỉnh trong tháng_y",
        "Vốn đăng ký trong tháng_y",
    ]
)
merge_df = merge_df.rename(
    columns={
        "Số dự án_x": "Số dự án",
        "Vốn đăng ký cấp mới_x": "Vốn đăng ký cấp mới",
        "Vốn đăng ký điều chỉnh_x": "Vốn đăng ký điều chỉnh",
        "Vốn đăng ký_x": "Vốn đăng ký",
        "Date_x": "Date",
        "Month_x": "Month",
        "PreMonth_x": "PreMonth",
        "PreYearMonth_x": "PreYearMonth",
        "Vùng Miền_x": "Vùng Miền",
        "Trong Nước_x": "Trong Nước",
        "Số dự án trong tháng_x": "Số dự án trong tháng",
        "Vốn đăng ký cấp mới trong tháng_x": "Vốn đăng ký cấp mới trong tháng",
        "Vốn đăng ký điều chỉnh trong tháng_x": "Vốn đăng ký điều chỉnh trong tháng",
        "Vốn đăng ký trong tháng_x": "Vốn đăng ký trong tháng",
    }
)
dfs_fdi = merge_df
merge_df[merge_df["Phân Loại"] == "TỔNG SỐ"]

,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date,Month,PreMonth,PreYearMonth,Vùng Miền,Trong Nước,Số dự án trong tháng,Vốn đăng ký cấp mới trong tháng,Vốn đăng ký điều chỉnh trong tháng,Vốn đăng ký trong tháng,Số dự án trong tháng - YOY,Vốn đăng ký cấp mới trong tháng - YOY,Vốn đăng ký điều chỉnh trong tháng - YOY,Vốn đăng ký trong tháng - YOY
0,TỔNG SỐ,2492,13553.757566,7643.956388,21197.713954,2024-09-29,2024-09,2024-08,2023-09,TỔNG SỐ,TỔNG SỐ,245,1555.257566,1931.256388,3486.513954,-25.757576,14.263068,212.311535,76.13148
40,TỔNG SỐ,2247,11998.5,5712.7,17711.2,2024-08-29,2024-08,2024-07,2023-08,TỔNG SỐ,TỔNG SỐ,431,1234.618278,744.700355,1979.318633,45.117845,31.746614,99.906907,51.134635
80,TỔNG SỐ,1816,10763.881722,4967.999645,15731.881367,2024-07-29,2024-07,2024-06,2023-07,TỔNG SỐ,TỔNG SỐ,278,1227.127617,1017.648916,2244.776533,-16.766467,-14.958722,-17.503377,-16.131502
120,TỔNG SỐ,1538,9536.754105,3950.350729,13487.104834,2024-06-29,2024-06,2024-05,2023-06,TỔNG SỐ,TỔNG SỐ,311,1600.512328,1868.006423,3468.518751,-6.042296,30.043188,189.022092,84.783267
160,TỔNG SỐ,1227,7936.241777,2082.344306,10018.586083,2024-05-29,2024-05,2024-04,2023-05,TỔNG SỐ,TỔNG SỐ,261,826.028911,847.715374,1673.744286,23.113208,-28.521053,36.66869,-5.752113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3634,TỔNG SỐ,907,NaN,NaN,7566.852767,2016-05-30,2016-05,2016-04,2015-05,TỔNG SỐ,TỔNG SỐ,210,NaN,NaN,2483.916421,NaN,NaN,NaN,NaN
3671,TỔNG SỐ,697,NaN,NaN,5082.936346,2016-04-29,2016-04,2016-03,2015-04,TỔNG SỐ,TỔNG SỐ,224,NaN,NaN,2342.490902,NaN,NaN,NaN,NaN
3708,TỔNG SỐ,473,NaN,NaN,2740.445444,2016-03-29,2016-03,2016-02,2015-03,TỔNG SỐ,TỔNG SỐ,182,NaN,NaN,835.347409,NaN,NaN,NaN,NaN
3743,TỔNG SỐ,291,NaN,NaN,1905.098035,2016-02-29,2016-02,2016-01,2015-02,TỔNG SỐ,TỔNG SỐ,164,NaN,NaN,893.698035,NaN,NaN,NaN,NaN


In [13]:
merge_df = pd.merge(
    left=dfs_fdi,
    right=dfs_fdi,
    left_on=["Phân Loại", "PreMonth"],
    right_on=["Phân Loại", "Month"],
    how="left",
)

merge_df["Số dự án trong tháng - MOM"] = np.where(
    merge_df["Số dự án trong tháng_y"] != 0,
    100
    * (
        merge_df["Số dự án trong tháng_x"]
        / (merge_df["Số dự án trong tháng_y"] + 1.0e-9)
    )
    - 100,
    np.nan,
)
merge_df["Vốn đăng ký cấp mới trong tháng - MOM"] = np.where(
    merge_df["Vốn đăng ký cấp mới trong tháng_y"] != 0,
    100
    * (
        merge_df["Vốn đăng ký cấp mới trong tháng_x"]
        / (merge_df["Vốn đăng ký cấp mới trong tháng_y"] + 1.0e-9)
    )
    - 100,
    np.nan,
)
merge_df["Vốn đăng ký điều chỉnh trong tháng - MOM"] = np.where(
    merge_df["Vốn đăng ký điều chỉnh trong tháng_y"] != 0,
    100
    * (
        merge_df["Vốn đăng ký điều chỉnh trong tháng_x"]
        / (merge_df["Vốn đăng ký điều chỉnh trong tháng_y"] + 1.0e-9)
    )
    - 100,
    np.nan,
)
merge_df["Vốn đăng ký trong tháng - MOM"] = np.where(
    merge_df["Vốn đăng ký trong tháng_y"] != 0,
    100
    * (
        merge_df["Vốn đăng ký trong tháng_x"]
        / (merge_df["Vốn đăng ký trong tháng_y"] + 1.0e-9)
    )
    - 100,
    np.nan,
)

merge_df = merge_df.drop(
    columns=[
        "Số dự án_y",
        "Vốn đăng ký cấp mới_y",
        "Vốn đăng ký điều chỉnh_y",
        "Vốn đăng ký_y",
        "Date_y",
        "Month_y",
        "PreMonth_y",
        "PreYearMonth_y",
        "Vùng Miền_y",
        "Trong Nước_y",
        "Số dự án trong tháng_y",
        "Vốn đăng ký cấp mới trong tháng_y",
        "Vốn đăng ký điều chỉnh trong tháng_y",
        "Vốn đăng ký trong tháng_y",
        "Số dự án trong tháng - YOY_y",
        "Vốn đăng ký cấp mới trong tháng - YOY_y",
        "Vốn đăng ký điều chỉnh trong tháng - YOY_y",
        "Vốn đăng ký trong tháng - YOY_y",
    ]
)
merge_df = merge_df.rename(
    columns={
        "Số dự án_x": "Số dự án",
        "Vốn đăng ký cấp mới_x": "Vốn đăng ký cấp mới",
        "Vốn đăng ký điều chỉnh_x": "Vốn đăng ký điều chỉnh",
        "Vốn đăng ký_x": "Vốn đăng ký",
        "Date_x": "Date",
        "Month_x": "Month",
        "PreMonth_x": "PreMonth",
        "PreYearMonth_x": "PreYearMonth",
        "Vùng Miền_x": "Vùng Miền",
        "Trong Nước_x": "Trong Nước",
        "Số dự án trong tháng_x": "Số dự án trong tháng",
        "Vốn đăng ký cấp mới trong tháng_x": "Vốn đăng ký cấp mới trong tháng",
        "Vốn đăng ký điều chỉnh trong tháng_x": "Vốn đăng ký điều chỉnh trong tháng",
        "Vốn đăng ký trong tháng_x": "Vốn đăng ký trong tháng",
        "Số dự án trong tháng - YOY_x": "Số dự án trong tháng - YOY",
        "Vốn đăng ký cấp mới trong tháng - YOY_x": "Vốn đăng ký cấp mới trong tháng - YOY",
        "Vốn đăng ký điều chỉnh trong tháng - YOY_x": "Vốn đăng ký điều chỉnh trong tháng - YOY",
        "Vốn đăng ký trong tháng - YOY_x": "Vốn đăng ký trong tháng - YOY",
    }
)
dfs_fdi = merge_df
merge_df[merge_df["Phân Loại"] == "TỔNG SỐ"]

,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date,Month,PreMonth,PreYearMonth,Vùng Miền,...,Vốn đăng ký điều chỉnh trong tháng,Vốn đăng ký trong tháng,Số dự án trong tháng - YOY,Vốn đăng ký cấp mới trong tháng - YOY,Vốn đăng ký điều chỉnh trong tháng - YOY,Vốn đăng ký trong tháng - YOY,Số dự án trong tháng - MOM,Vốn đăng ký cấp mới trong tháng - MOM,Vốn đăng ký điều chỉnh trong tháng - MOM,Vốn đăng ký trong tháng - MOM
0,TỔNG SỐ,2492,13553.757566,7643.956388,21197.713954,2024-09-29,2024-09,2024-08,2023-09,TỔNG SỐ,...,1931.256388,3486.513954,-25.757576,14.263068,212.311535,76.13148,-43.155452,25.970723,159.333351,76.14718
40,TỔNG SỐ,2247,11998.5,5712.7,17711.2,2024-08-29,2024-08,2024-07,2023-08,TỔNG SỐ,...,744.700355,1979.318633,45.117845,31.746614,99.906907,51.134635,55.035971,0.610422,-26.821486,-11.825582
80,TỔNG SỐ,1816,10763.881722,4967.999645,15731.881367,2024-07-29,2024-07,2024-06,2023-07,TỔNG SỐ,...,1017.648916,2244.776533,-16.766467,-14.958722,-17.503377,-16.131502,-10.610932,-23.329074,-45.522194,-35.281407
120,TỔNG SỐ,1538,9536.754105,3950.350729,13487.104834,2024-06-29,2024-06,2024-05,2023-06,TỔNG SỐ,...,1868.006423,3468.518751,-6.042296,30.043188,189.022092,84.783267,19.157088,93.759844,120.357738,107.231103
160,TỔNG SỐ,1227,7936.241777,2082.344306,10018.586083,2024-05-29,2024-05,2024-04,2023-05,TỔNG SỐ,...,847.715374,1673.744286,23.113208,-28.521053,36.66869,-5.752113,-18.944099,-64.642253,182.54407,-36.509999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3789,TỔNG SỐ,907,NaN,NaN,7566.852767,2016-05-30,2016-05,2016-04,2015-05,TỔNG SỐ,...,NaN,2483.916421,NaN,NaN,NaN,NaN,-6.25,NaN,NaN,6.037399
3826,TỔNG SỐ,697,NaN,NaN,5082.936346,2016-04-29,2016-04,2016-03,2015-04,TỔNG SỐ,...,NaN,2342.490902,NaN,NaN,NaN,NaN,23.076923,NaN,NaN,180.421161
3863,TỔNG SỐ,473,NaN,NaN,2740.445444,2016-03-29,2016-03,2016-02,2015-03,TỔNG SỐ,...,NaN,835.347409,NaN,NaN,NaN,NaN,10.97561,NaN,NaN,-6.529121
3898,TỔNG SỐ,291,NaN,NaN,1905.098035,2016-02-29,2016-02,2016-01,2015-02,TỔNG SỐ,...,NaN,893.698035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
dfs_fdi.to_csv('../csv/fdi.csv')

In [15]:
nganh = "TỔNG SỐ"

dfs_fdi_hanghoa = dfs_fdi[dfs_fdi["Phân Loại"] == nganh]
dfs_fdi.loc[dfs_fdi["Phân Loại"] == nganh, "Vốn đăng ký so với cùng kỳ"] = dfs_fdi_hanghoa["Vốn đăng ký"].pct_change(-12)
dfs_fdi

/tmp/ipykernel_23052/4230591522.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfs_fdi.loc[dfs_fdi["Phân Loại"] == nganh, "Vốn đăng ký so với cùng kỳ"] = dfs_fdi_hanghoa["Vốn đăng ký"].pct_change(-12)


,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date,Month,PreMonth,PreYearMonth,Vùng Miền,...,Vốn đăng ký trong tháng,Số dự án trong tháng - YOY,Vốn đăng ký cấp mới trong tháng - YOY,Vốn đăng ký điều chỉnh trong tháng - YOY,Vốn đăng ký trong tháng - YOY,Số dự án trong tháng - MOM,Vốn đăng ký cấp mới trong tháng - MOM,Vốn đăng ký điều chỉnh trong tháng - MOM,Vốn đăng ký trong tháng - MOM,Vốn đăng ký so với cùng kỳ
0,TỔNG SỐ,2492,13553.757566,7643.956388,21197.713954,2024-09-29,2024-09,2024-08,2023-09,TỔNG SỐ,...,3486.513954,-25.757576,14.263068,212.311535,76.13148,-43.155452,25.970723,159.333351,76.14718,0.377926
1,Bà Rịa - Vũng Tàu,34,1681.168637,-9.557457,1671.61118,2024-09-29,2024-09,2024-08,2023-09,Miền Nam,...,29.6,NaN,NaN,NaN,NaN,-60.0,-75.005804,NaN,-75.005804,NaN
2,Bắc Ninh,325,1613.384153,2644.33862,4257.722773,2024-09-29,2024-09,2024-08,2023-09,Miền Bắc,...,1042.274973,-85.106383,-80.342581,1229.606354,608.774323,-88.135593,-91.215416,1014.69289,319.883914,NaN
3,Quảng Ninh,27,1609.642987,203.00106,1812.644047,2024-09-29,2024-09,2024-08,2023-09,Miền Bắc,...,35.063265,NaN,NaN,NaN,NaN,100.0,-83.661974,-100.0,-83.67739,NaN
4,Hà Nội,208,1121.315759,179.449427,1300.765185,2024-09-29,2024-09,2024-08,2023-09,Miền Bắc,...,33.885896,-30.232558,-87.640367,-19.246684,-79.934255,-14.285714,-31.388196,-39.034022,-35.080009,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3960,Quần đảo Vigin thuộc Anh,3,NaN,NaN,9.4,2016-01-29,2016-01,2015-12,2015-01,Châu Mỹ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3961,Đài Loan,6,NaN,NaN,8.7,2016-01-29,2016-01,2015-12,2015-01,Châu Á,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3962,Thái Lan,3,NaN,NaN,6.3,2016-01-29,2016-01,2015-12,2015-01,Châu Á,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3963,Hà Lan,1,NaN,NaN,6,2016-01-29,2016-01,2015-12,2015-01,Châu Âu,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_fdi[(dfs_fdi["Phân Loại"] == "TỔNG SỐ")]

fig = px.area(df_show, x="Date", y="Vốn đăng ký so với cùng kỳ", color="Phân Loại")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()